In [1]:
import os
import pdfplumber
import pandas as pd
import re

# Function to extract specific text from PDF
def extract_info_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    
    # Define regex patterns for information extraction
    name_pattern = r"PERSONNE ETROITEMENT LIEE :(.*?)(?=,|$)"
    position_pattern = r",(.*?)(?=NOTIFICATION|$)"
    company_name_pattern = r"NOM :(.*?)(?=LEI|DETAIL|$)" # LEI or DETAIL to stop
    date_pattern = r"DATE DE LA TRANSACTION :(.*?)(?=LIEU|$)"
    nature_pattern = r"NATURE DE LA TRANSACTION :(.*?)(?=DESCRIPTION|$)"
    price_pattern = r"PRIX :(.*?)(?=Euro|Dollar|Livre|Franc|$)"
    volume_pattern = r"VOLUME :((?:(?!VOLUME :|TRANSACTION).)*)\s*TRANSACTION" # last occurence
    date_notification_pattern = r"DATE DE RECEPTION DE LA NOTIFICATION :(.*?)(?=COMMENTAIRES|$)"

    name = re.search(name_pattern, text, re.DOTALL)
    position = re.search(position_pattern, text, re.DOTALL)
    company_name = re.search(company_name_pattern, text, re.DOTALL)
    date_transaction = re.search(date_pattern, text, re.DOTALL)
    nature = re.search(nature_pattern, text, re.DOTALL)
    price = re.search(price_pattern, text, re.DOTALL)
    volume = re.search(volume_pattern, text, re.DOTALL)
    date_notification = re.search(date_notification_pattern, text, re.DOTALL)

    # Extracted information
    def convert_to_text(re_search_result):
        return re_search_result.group(1).strip() if re_search_result else ""
    
    name_text = convert_to_text(name)
    position_text = convert_to_text(position)
    company_name_text = convert_to_text(company_name)
    date_transaction_text = convert_to_text(date_transaction)
    nature_text = convert_to_text(nature)
    price_text = convert_to_text(price)
    volume_text = convert_to_text(volume)
    date_notification_text = convert_to_text(date_notification)

    text_dict = {'name': name_text, 'position': position_text, 'company_name': company_name_text,
                 'date_transaction': date_transaction_text, 'date_notification': date_notification_text,
                 'nature': nature_text, 'price': price_text, 'volume': volume_text}
    
    return text_dict

folder_path = "./amf-pdfs/"

# Find all PDF files in the folder
pdf_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".pdf")]
print(f'list of PDF files: {pdf_files}')
print(f'number of PDF files: {len(pdf_files)}')

data = []

# Extract information from each PDF and store it in the dataframe
for file in pdf_files:
    new_info_dict = extract_info_from_pdf(file)
    data.append(new_info_dict)
df = pd.DataFrame(data)

df.head(10)

list of PDF files: ['./amf-pdfs/DD_22_848579_9241245.pdf', './amf-pdfs/DD_22_873197_9545767.pdf', './amf-pdfs/DD_22_873757_9553231.pdf', './amf-pdfs/DD_22_815707_8827457.pdf', './amf-pdfs/DD_22_877789_9603925.pdf', './amf-pdfs/DD_22_833404_9052280.pdf', './amf-pdfs/DD_22_824970_8938825.pdf', './amf-pdfs/DD_22_876817_9591571.pdf', './amf-pdfs/DD_22_834822_9070591.pdf', './amf-pdfs/DD_22_865073_9445323.pdf', './amf-pdfs/DD_22_842912_9173472.pdf', './amf-pdfs/DD_22_845445_9204231.pdf', './amf-pdfs/DD_22_842688_9170905.pdf', './amf-pdfs/DD_22_819906_8878517.pdf', './amf-pdfs/DD_22_854540_9315544.pdf', './amf-pdfs/DD_22_858763_9367927.pdf', './amf-pdfs/DD_22_826672_8958867.pdf', './amf-pdfs/DD_22_843087_9175557.pdf', './amf-pdfs/DD_22_843559_9181314.pdf', './amf-pdfs/DD_22_850873_9269347.pdf', './amf-pdfs/DD_22_841384_9155091.pdf', './amf-pdfs/DD_22_867041_9468981.pdf', './amf-pdfs/DD_22_816876_8843228.pdf', './amf-pdfs/DD_22_825465_8944800.pdf', './amf-pdfs/DD_22_855424_9326881.pdf', './am

,name,position,company_name,date_transaction,date_notification,nature,price,volume
0,James DeYonker,Directeur Juridique,EUROAPI,20 juin 2022,23 juin 2022,Acquisition,14.4000,416.0000
1,ARJIL COMMANDITEE-ARCO SOCIETE ANONYME personn...,PRESIDENT DIRECTEUR\nGENERAL,LAGARDERE SA,22 novembre 2022,23 novembre 2022,Acquisition,18.9857,1 860.0000
2,The Home Bar Bevtech LTD personne morale liée ...,Président Directeur Général,SMART GOOD THINGS HOLDING S.A.,28 novembre 2022,28 novembre 2022,Cession,158.0000,265.0000
3,HERITAGE VENTURES LIMITED Private Company Limi...,Shareholder,AIRTIME PARTECIPAZIONI S.P.A.,11 janvier 2022,11 janvier 2022,Cession,74.8000,135.0000
4,Paul DU SAILLANT,Directeur Général Délégué,ESSILORLUXOTTICA,23 décembre 2022,29 décembre 2022,Cession,169.8530,8 831.7470
5,Marion Alary personne liée à Jean-Paul Alary,Président de Safran Aircraft Engines,SAFRAN,31 mars 2022,01 avril 2022,Réception d'une donation,108.3000,1 846.0000
6,BOUYGUES SA,"ADMINISTRATEUR DE TF1, FILIALE DIRECTE DE BOUY...",TELEVISION FRANCAISE 1,04 mars 2022,07 mars 2022,Acquisition,8.1588,60 000.0000
7,Nicolas HIERONIMUS,Directeur Général et Administrateur,L'OREAL,16 décembre 2022,19 décembre 2022,DONATION DE LA NUE PROPRIETE EFFECTUEE,0.0000,350 994.0000
8,DB2 SARL personne morale liée à Christophe de ...,Administrateur,REALITES,07 avril 2022,08 avril 2022,Cession,35.5000,13.0000
9,paul lorne,Directeur Général Délégué,SPARTOO,04 octobre 2022,05 octobre 2022,Acquisition,1.2300,4 470.0000


In [2]:
from datetime import datetime
import locale

# Set the French locale for month names
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Convert the 'date' column to datetime format
df['date_transaction'] = pd.to_datetime(df['date_transaction'], format='%d %B %Y', errors='coerce')
df['date_notification'] = pd.to_datetime(df['date_notification'], format='%d %B %Y', errors='coerce')

# Reset the locale to the default setting
locale.setlocale(locale.LC_TIME, '')

df.head(10)


,name,position,company_name,date_transaction,date_notification,nature,price,volume
0,James DeYonker,Directeur Juridique,EUROAPI,2022-06-20,2022-06-23,Acquisition,14.4000,416.0000
1,ARJIL COMMANDITEE-ARCO SOCIETE ANONYME personn...,PRESIDENT DIRECTEUR\nGENERAL,LAGARDERE SA,2022-11-22,2022-11-23,Acquisition,18.9857,1 860.0000
2,The Home Bar Bevtech LTD personne morale liée ...,Président Directeur Général,SMART GOOD THINGS HOLDING S.A.,2022-11-28,2022-11-28,Cession,158.0000,265.0000
3,HERITAGE VENTURES LIMITED Private Company Limi...,Shareholder,AIRTIME PARTECIPAZIONI S.P.A.,2022-01-11,2022-01-11,Cession,74.8000,135.0000
4,Paul DU SAILLANT,Directeur Général Délégué,ESSILORLUXOTTICA,2022-12-23,2022-12-29,Cession,169.8530,8 831.7470
5,Marion Alary personne liée à Jean-Paul Alary,Président de Safran Aircraft Engines,SAFRAN,2022-03-31,2022-04-01,Réception d'une donation,108.3000,1 846.0000
6,BOUYGUES SA,"ADMINISTRATEUR DE TF1, FILIALE DIRECTE DE BOUY...",TELEVISION FRANCAISE 1,2022-03-04,2022-03-07,Acquisition,8.1588,60 000.0000
7,Nicolas HIERONIMUS,Directeur Général et Administrateur,L'OREAL,2022-12-16,2022-12-19,DONATION DE LA NUE PROPRIETE EFFECTUEE,0.0000,350 994.0000
8,DB2 SARL personne morale liée à Christophe de ...,Administrateur,REALITES,2022-04-07,2022-04-08,Cession,35.5000,13.0000
9,paul lorne,Directeur Général Délégué,SPARTOO,2022-10-04,2022-10-05,Acquisition,1.2300,4 470.0000


In [3]:
# Ensure Price and Volume are Float values
df['price'] = df['price'].astype(str).str.replace(' ', '').astype(float)
df['volume'] = df['volume'].astype(str).str.replace(' ', '').astype(float)
df.tail(10)

,name,position,company_name,date_transaction,date_notification,nature,price,volume
4896,SMAvie BTP Société d'assurances mutuelles pers...,Censeur,LNA SANTE,2022-05-10,2022-06-08,Acquisition,34.9840,298.0
4897,STEPHANE GERART,CIO - Directeur BU IA - Membre du comex,ONCODESIGN,2022-10-18,2022-10-20,Cession,14.4200,17640.0
4898,ISABELLE SIMON,Secrétaire général,THALES,2022-09-28,2022-09-28,Acquisition gratuite d'actions,0.0000,2493.0
4899,BOUYGUES SA,"ADMINISTRATEUR DE TF1, FILIALE DIRECTE DE BOUY...",TELEVISION FRANCAISE 1,2022-08-16,2022-08-17,Acquisition,6.7872,10000.0
4900,HOLDING HR Société par actions simplifiée pers...,Président Directeur Général,HYDROGEN-REFUELING-SOLUTIONS SA,2022-02-08,2022-02-10,Cession,34.8650,102108.0
4901,BOUYGUES SA,"ADMINISTRATEUR DE TF1, FILIALE DIRECTE DE BOUY...",TELEVISION FRANCAISE 1,2022-05-05,2022-05-06,Acquisition,7.9532,34000.0
4902,Aymerick PENICAUT,PDG,ASHLER & MANSON,2022-08-17,2022-08-19,Cession,2.8400,242.0
4903,CAMAHËAL Finance S.A. personne morale liée à J...,PDG,ALAN ALLMAN ASSOCIATES,2022-01-04,2022-01-07,Cession,15.2172,1638.0
4904,Kerry-Jane Wallart personne liée à Romain Desr...,Directeur Général Délégué,NEOEN,2022-06-23,2022-06-27,Paiement dividende en actions,33.9900,78.0
4905,Eric BAISSUS,President du Directoire de Kalray,KALRAY,2022-08-10,2022-10-18,Cession,23.1546,2500.0


In [4]:
# company_names = df['company_name'].unique()

# ticker_mapping = {}

# for company_name in company_names:
#     ticker_mapping[company_name] = '.PA'

# ticker_mapping = dict(sorted(ticker_mapping.items()))

# print(ticker_mapping)

import ticker_mapping
ticker_mapping = ticker_mapping.ticker_mapping

In [5]:
company_names = df['company_name'].unique()
missing_keys = [company_name for company_name in company_names if company_name not in ticker_mapping]
print(f'missing keys: {missing_keys}')

df['ticker'] = df['company_name'].map(ticker_mapping)

df.head(15)

missing keys: []


,name,position,company_name,date_transaction,date_notification,nature,price,volume,ticker
0,James DeYonker,Directeur Juridique,EUROAPI,2022-06-20,2022-06-23,Acquisition,14.4000,416.000,EAPI.PA
1,ARJIL COMMANDITEE-ARCO SOCIETE ANONYME personn...,PRESIDENT DIRECTEUR\nGENERAL,LAGARDERE SA,2022-11-22,2022-11-23,Acquisition,18.9857,1860.000,MMB.PA
2,The Home Bar Bevtech LTD personne morale liée ...,Président Directeur Général,SMART GOOD THINGS HOLDING S.A.,2022-11-28,2022-11-28,Cession,158.0000,265.000,MLSGT.PA
3,HERITAGE VENTURES LIMITED Private Company Limi...,Shareholder,AIRTIME PARTECIPAZIONI S.P.A.,2022-01-11,2022-01-11,Cession,74.8000,135.000,ALMEX.PA
4,Paul DU SAILLANT,Directeur Général Délégué,ESSILORLUXOTTICA,2022-12-23,2022-12-29,Cession,169.8530,8831.747,EL.PA
5,Marion Alary personne liée à Jean-Paul Alary,Président de Safran Aircraft Engines,SAFRAN,2022-03-31,2022-04-01,Réception d'une donation,108.3000,1846.000,SAF.PA
6,BOUYGUES SA,"ADMINISTRATEUR DE TF1, FILIALE DIRECTE DE BOUY...",TELEVISION FRANCAISE 1,2022-03-04,2022-03-07,Acquisition,8.1588,60000.000,TFI.PA
7,Nicolas HIERONIMUS,Directeur Général et Administrateur,L'OREAL,2022-12-16,2022-12-19,DONATION DE LA NUE PROPRIETE EFFECTUEE,0.0000,350994.000,OR.PA
8,DB2 SARL personne morale liée à Christophe de ...,Administrateur,REALITES,2022-04-07,2022-04-08,Cession,35.5000,13.000,ALREA.PA
9,paul lorne,Directeur Général Délégué,SPARTOO,2022-10-04,2022-10-05,Acquisition,1.2300,4470.000,ALSPT.PA


In [6]:
import yfinance as yf
from datetime import timedelta

# Get current and future stock prices and variations
def get_stock_prices(row):
    try:
        ticker = row['ticker']
        start_day = row['date_transaction']
        end_day = start_day + timedelta(days=90)
        start_day = start_day.strftime('%Y-%m-%d')
        end_day = end_day.strftime('%Y-%m-%d')

        stock = yf.Ticker(ticker)
        stock_data = stock.history(start=start_day, end=end_day)
        prices = stock_data['Open']
        prices_dict = {
            'stock_price_open_d0': prices.values[0],
            'var_d1': prices.values[1] / prices.values[0],
            'var_d3': prices.values[3] / prices.values[0],
            'var_d5': prices.values[5] / prices.values[0],
            'var_d10': prices.values[10] / prices.values[0],
            'var_d20': prices.values[20] / prices.values[0],
            'var_d30': prices.values[30] / prices.values[0],
            'var_d60': prices.values[60] / prices.values[0]
        }
        return pd.Series(prices_dict)
    except:
        return {
            'stock_price_open_d0': float('nan'),
            'var_d1': float('nan'),
            'var_d3': float('nan'),
            'var_d5': float('nan'),
            'var_d10': float('nan'),
            'var_d20': float('nan'),
            'var_d30': float('nan'),
            'var_d60': float('nan')
        }

# Applying the function to new columns
df = pd.concat([df, df.apply(get_stock_prices, axis=1)], axis=1)

df.head(15)

df.to_pickle('amf-data.pkl')

- UMS.PA: No data found for this date range, symbol may be delisted
- URW.PA: Data doesn't exist for startDate = 1649196000, endDate = 1656972000
- EDF.PA: No data found, symbol may be delisted
- ???.PA: No data found for this date range, symbol may be delisted
- TCHCS.PA: Data doesn't exist for startDate = 1650319200, endDate = 1658095200
- ???.PA: No data found for this date range, symbol may be delisted
- ???.PA: No data found for this date range, symbol may be delisted
- URW.PA: Data doesn't exist for startDate = 1647817200, endDate = 1655589600
- UMS.PA: No data found for this date range, symbol may be delisted
- URW.PA: Data doesn't exist for startDate = 1646607600, endDate = 1654380000
- TCHCS.PA: Data doesn't exist for startDate = 1646089200, endDate = 1653861600
- ???.PA: No data found for this date range, symbol may be delisted
- ???.PA: No data found for this date range, symbol may be delisted
- CNPAP.PA: No data found for this date range, symbol may be delisted
- ???.PA: No

In [19]:
import pandas as pd

# Test if cession and acquisition future variation are going up or down
amf_data = pd.read_pickle('amf-data.pkl')
amf_df = pd.DataFrame(amf_data)

amf_df = amf_df[(amf_df['date_transaction'] >= '2022-01-01') & (amf_df['date_transaction'] <= '2022-04-30')]

amf_df = amf_df[amf_df['price'] != 0]

amf_df = amf_df[amf_df['position'].isin(['Président Directeur Général', 'PDG', 'Président-Directeur Général', 'président-directeur général', 'DIRECTEUR', 'Directeur Général', 'DIRIGEANT', 'Directeur général', 'P.D.-G.'])]

print(f'data length: {len(amf_df)}')

df_cession = amf_df[amf_df['nature'] == 'Cession']
# df_acquisition = amf_df[amf_df['nature'] == 'Acquisition']
df_acquisition = amf_df[amf_df['nature'] != 'Cession']

var_means = {
    'cession': {
        'd1': df_cession['var_d1'].mean(),
        'd3': df_cession['var_d3'].mean(),
        'd5': df_cession['var_d5'].mean(),
        'd10': df_cession['var_d10'].mean(),
        'd20': df_cession['var_d20'].mean(),
        'd30': df_cession['var_d30'].mean(),
        'd60': df_cession['var_d60'].mean(),
    },
    'acquisition' : {
        'd1': df_acquisition['var_d1'].mean(),
        'd3': df_acquisition['var_d3'].mean(),
        'd5': df_acquisition['var_d5'].mean(),
        'd10': df_acquisition['var_d10'].mean(),
        'd20': df_acquisition['var_d20'].mean(),
        'd30': df_acquisition['var_d30'].mean(),
        'd60': df_acquisition['var_d60'].mean(),
    }
}

var_medians = {
    'cession': {
        'd1': df_cession['var_d1'].median(),
        'd3': df_cession['var_d3'].median(),
        'd5': df_cession['var_d5'].median(),
        'd10': df_cession['var_d10'].median(),
        'd20': df_cession['var_d20'].median(),
        'd30': df_cession['var_d30'].median(),
        'd60': df_cession['var_d60'].median()
    },
    'acquisition' : {
        'd1': df_acquisition['var_d1'].median(),
        'd3': df_acquisition['var_d3'].median(),
        'd5': df_acquisition['var_d5'].median(),
        'd10': df_acquisition['var_d10'].median(),
        'd20': df_acquisition['var_d20'].median(),
        'd30': df_acquisition['var_d30'].median(),
        'd60': df_acquisition['var_d60'].median()
    }
}

var_means = {key: {inner_key: round(inner_value, 2) for inner_key, inner_value in value.items()} for key, value in var_means.items()}
var_medians = {key: {inner_key: round(inner_value, 2) for inner_key, inner_value in value.items()} for key, value in var_medians.items()}

print(var_means)
print(var_medians)

data length: 119
{'cession': {'d1': 1.01, 'd3': 0.99, 'd5': 0.99, 'd10': 0.97, 'd20': 0.97, 'd30': 0.97, 'd60': 0.96}, 'acquisition': {'d1': 1.01, 'd3': 1.02, 'd5': 1.02, 'd10': 1.02, 'd20': 1.0, 'd30': 0.98, 'd60': 0.95}}
{'cession': {'d1': 1.01, 'd3': 1.0, 'd5': 0.99, 'd10': 0.98, 'd20': 0.98, 'd30': 0.99, 'd60': 1.0}, 'acquisition': {'d1': 1.0, 'd3': 1.0, 'd5': 1.0, 'd10': 1.0, 'd20': 1.0, 'd30': 0.99, 'd60': 0.96}}


In [13]:
import pandas as pd

# Test if cession and acquisition future variation are going up or down
amf_data = pd.read_pickle('amf-data.pkl')
amf_df = pd.DataFrame(amf_data)
amf_df = amf_df.sort_values(by='var_d5', ascending=False)

pivot_table = pd.pivot_table(amf_df, values=['var_d1', 'var_d3', 'var_d5', 'var_d10', 'var_d20', 'var_d30', 'company_name'],
                       index=['position', 'nature'], aggfunc={'var_d1': 'median',
                                                    'var_d3': 'median',
                                                    'var_d5': ['median', 'count'],
                                                    'var_d10': 'median',
                                                    'var_d20': 'median',
                                                    'var_d30': 'median',
                                                    'company_name': 'nunique'})
pivot_table = pivot_table.sort_values(by=('var_d3', 'median'), ascending=False)

pivot_table2 = pd.pivot_table(amf_df, values=['var_d1', 'var_d3', 'var_d5', 'var_d10', 'var_d20', 'var_d30'],
                       index=['company_name', 'nature'], aggfunc={'var_d1': 'median',
                                                    'var_d3': 'median',
                                                    'var_d5': ['median', 'count'],
                                                    'var_d10': 'median',
                                                    'var_d20': 'median',
                                                    'var_d30': 'median'})
pivot_table2 = pivot_table2.sort_values(by=('var_d5', 'median'), ascending=False)

pivot_table3 = pd.pivot_table(amf_df, values=['var_d1', 'var_d3', 'var_d5', 'var_d10', 'var_d20', 'var_d30'],
                       index=['name', 'company_name', 'nature'], aggfunc={'var_d1': 'median',
                                                    'var_d3': 'median',
                                                    'var_d5': ['median', 'count'],
                                                    'var_d10': 'median',
                                                    'var_d20': 'median',
                                                    'var_d30': 'median'})
pivot_table3 = pivot_table3.sort_values(by=('var_d5', 'median'), ascending=False)

pd.set_option('display.max_rows', None)  # Show all rows
# amf_df.head(200)
pivot_table3.head(1000)


var_d1  \
                                                                                                         median   
name                                               nature                                                         
P.C.G. société civile personne morale liée à PH... Acquisition                                         1.207619   
COMPAGNIE FINANCIERE DE PRISES DE PARTICIPATION... Acquisition                                         1.000000   
SERGE BITBOUL                                      Acquisition                                         1.573529   
XLP HOLDING SAS personne morale liée à SERGE BI... Acquisition                                         1.573529   
Jan HOFLACK                                        Acquisition                                         1.031746   
HERITAGE VENTURES LIMITED Private Company Limit... Acquisition                                         1.190722   
Yseulys COSTES                                     Nantissement                                        1.256410   
Thibaut Munier                                     Nantissement                                        1.256410   
ARTEME                                             Acquisition                                         1.182482   
Julien Parrou personne liée à Saint-Florentin P... Acquisition                                         1.024742   
Gérard GUILLEMOT                                   Acquisition                                         1.042105   
Nami Itoh-Korsholm                                 Acquisition                                         0.981211   
AFI.ESCA SA personne morale liée à CHRISTIAN BU... Acquisition                                         1.091346   
JOEL PICARD                                        Souscription                                        1.053659   
NORTHBROOK BELGIUM Société anonyme personne mor... Souscription                                        1.053659   
Christophe GAUSSIN                                 Acquisition                                         1.169552   
Jean-Claude Bailly                                 Aquisition d'Actions Gratuites                      1.125397   
Olivier CRAMATTE                                   Acquisition                                         1.125397   
Aziz Azougagh                                      Acquisition                                         1.125397   
William Lee                                        definitive acquisition of the free share allocated  1.125397   
Gary Patterson                                     Acquisition of free shares”                         1.125397   
Gilles GRAPINET                                    Acquisition                                         1.010723   
Marc-Henri DESPORTES                               Acquisition                                         1.010723   
KER VENTURES EURL SARL personne morale liée à F... Prêt                                                0.969444   
GEMMES VENTURE personne morale liée à philippe ... Acquisition                                         1.061189   
Cyrille Tupin                                      Acquisition                                         1.095181   
ARTEME                                             Cession                                             1.038044   
Philippe Guillemot                                 Acquisition                                         1.045675   
HERITAGE VENTURES LIMITED Private Company Limit... Cession                                             1.200000   
Nicolas Dubois                                     Acquisition                                         0.989286   
sandy bourdin                                      Acquisition                                         0.989286   
Yves-Marie Gille                                   Acquisition                                         0.989286   
Saloua Maslaga                                     Acquisition                  

In [12]:
selection_df = amf_df[amf_df['company_name'] == 'AMA CORPORATION PLC']
selection_df.sort_values(by='var_d3', ascending=False)
selection_df.head(50)

,name,position,company_name,date_transaction,date_notification,nature,price,volume,ticker,stock_price_open_d0,var_d1,var_d3,var_d5,var_d10,var_d20,var_d30,var_d60
1862,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-11,2022-05-13,Acquisition,1.6400,7861.0,ALAMA.PA,1.540,1.087662,1.233766,1.389610,1.480519,0.876623,0.454545,0.481818
4265,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-13,2022-05-16,Acquisition,1.8849,15980.0,ALAMA.PA,1.720,1.104651,1.244186,1.372093,1.136628,0.732558,0.338372,0.429070
4328,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-12,2022-05-13,Acquisition,1.7013,2371.0,ALAMA.PA,1.675,1.026866,1.182090,1.355224,1.194030,0.740299,0.382090,0.437015
2336,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-10,2022-05-13,Acquisition,1.5459,18743.0,ALAMA.PA,1.510,1.019868,1.139073,1.311258,1.556291,0.966887,0.582781,0.478146
639,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-16,2022-05-16,Acquisition,2.0022,5045.0,ALAMA.PA,1.900,1.042105,1.194737,1.273684,1.010526,0.610526,0.294737,0.378947
1085,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-09,2022-05-10,Acquisition,1.5391,5789.0,ALAMA.PA,1.515,0.996700,1.105611,1.254125,1.597360,0.933993,0.592739,0.508251
1194,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-05,2022-05-09,Acquisition,1.3747,12631.0,ALAMA.PA,1.345,1.048327,1.122677,1.245353,1.687732,1.159851,0.698885,0.709294
2598,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-06,2022-05-09,Acquisition,1.4741,3619.0,ALAMA.PA,1.410,1.074468,1.092199,1.219858,1.673759,1.063830,0.646809,0.624113
4019,Nami Itoh-Korsholm,Director,AMA CORPORATION PLC,2021-11-03,2022-06-16,Acquisition,0.0000,10000.0,ALAMA.PA,2.395,0.981211,1.073069,1.212944,1.308977,1.018789,0.918580,0.818372
103,Gérard GUILLEMOT,DIRECTEUR,AMA CORPORATION PLC,2022-05-17,2022-05-19,Acquisition,2.0525,9748.0,ALAMA.PA,1.980,1.080808,1.191919,1.186869,1.002525,0.510101,0.282828,0.363636
